# Convert Model to TensorFlow Lite

This notebook loads the trained Keras model, converts it into the TensorFlow Lite (`.tflite`) format for edge deployment, and demonstrates how to create a quantized version for further optimization.


In [6]:
import tensorflow as tf
import os
import joblib # To load the label encoder
import numpy as np
print(f"TensorFlow Version: {tf.__version__}")


TensorFlow Version: 2.20.0


## 1. Load the Trained Keras Model

First, we load the complete model that we saved in the training notebook. This includes the model architecture and its trained weights.


In [7]:
# --- Paths to saved model and encoder ---
MODEL_PATH = "water_leak_model.h5"
LABEL_ENCODER_PATH = "label_encoder.joblib"

# --- Load the model ---
if os.path.exists(MODEL_PATH):
    # When loading a model with custom layers or functions, you might need a custom_objects dictionary.
    # Our model uses standard layers, so it's not needed here.
    model = tf.keras.models.load_model(MODEL_PATH)
    print("Keras model loaded successfully.")
    model.summary()
else:
    print(f"Error: Model file not found at '{MODEL_PATH}'")
    model = None

# --- Load the label encoder ---
if os.path.exists(LABEL_ENCODER_PATH):
    label_encoder = joblib.load(LABEL_ENCODER_PATH)
    print(f"Label encoder loaded successfully. Classes: {label_encoder.classes_}")
else:
    print(f"Error: Label encoder file not found at '{LABEL_ENCODER_PATH}'")
    label_encoder = None


Keras model loaded successfully.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 429, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 214, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 212, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 106, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 104, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 52, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,315 (427.02 KB)

 Trainable params: 109,313 (427.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

Label encoder loaded successfully. Classes: ['Leak' 'NoLeak']


## 2. Convert to Standard TensorFlow Lite Model

Now, we use the `TFLiteConverter` to create a standard `.tflite` model. This version uses 32-bit floats for its weights.


In [8]:
if model:
    # --- Convert the model ---
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    # --- Save the TFLite model ---
    TFLITE_MODEL_PATH = "water_leak_model.tflite"
    with open(TFLITE_MODEL_PATH, 'wb') as f:
        f.write(tflite_model)
    
    print(f"Standard TFLite model saved to: {TFLITE_MODEL_PATH}")
    print(f"File size: {os.path.getsize(TFLITE_MODEL_PATH) / 1024:.2f} KB")
else:
    print("Model not loaded, skipping TFLite conversion.")


INFO:tensorflow:Assets written to: C:\Users\fe_de\AppData\Local\Temp\tmpaicfkb28\assets


INFO:tensorflow:Assets written to: C:\Users\fe_de\AppData\Local\Temp\tmpaicfkb28\assets


Saved artifact at 'C:\Users\fe_de\AppData\Local\Temp\tmpaicfkb28'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 431, 1), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2515370090256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370091600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370090832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370091216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370094672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370095440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370092176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370095824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370096016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370096208: TensorSpec(shape=(), dtype=tf.resource, name=None)
Standa

## 3. Convert with Float16 Quantization

Quantization is a technique to reduce model size and speed up inference. Here, we'll use **Float16 quantization**, which halves the model size by converting the weights from 32-bit floats to 16-bit floats with minimal loss in accuracy. This is a great first optimization step for edge devices.


In [10]:
if model:
    # --- Configure the converter for Float16 quantization ---
    converter_fp16 = tf.lite.TFLiteConverter.from_keras_model(model)
    converter_fp16.optimizations = [tf.lite.Optimize.DEFAULT]
    converter_fp16.target_spec.supported_types = [tf.float16]
    
    # --- Convert the model ---
    tflite_model_fp16 = converter_fp16.convert()

    # --- Save the quantized TFLite model ---
# ... existing code ...
    TFLITE_FP16_MODEL_PATH = "water_leak_model_fp16.tflite"
    with open(TFLITE_FP16_MODEL_PATH, 'wb') as f:
        f.write(tflite_model_fp16)
        
    print(f"Float16 quantized TFLite model saved to: {TFLITE_FP16_MODEL_PATH}")
    print(f"File size: {os.path.getsize(TFLITE_FP16_MODEL_PATH) / 1024:.2f} KB")
else:
    print("Model not loaded, skipping TFLite conversion.")


INFO:tensorflow:Assets written to: C:\Users\fe_de\AppData\Local\Temp\tmpir8vjkt1\assets


INFO:tensorflow:Assets written to: C:\Users\fe_de\AppData\Local\Temp\tmpir8vjkt1\assets


Saved artifact at 'C:\Users\fe_de\AppData\Local\Temp\tmpir8vjkt1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 431, 1), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2515370090256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370091600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370090832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370091216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370094672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370095440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370092176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370095824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370096016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2515370096208: TensorSpec(shape=(), dtype=tf.resource, name=None)
Float1

## 4. Test the Converted Models

To ensure the conversion was successful, we can load the `.tflite` models and run a test inference on a dummy input. This verifies that the models are loadable and can produce an output.


In [11]:
def test_tflite_model(model_path):
    if not os.path.exists(model_path):
        print(f"File not found: {model_path}")
        return

    # Load the TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Create a dummy input tensor that matches the model's expected input shape
    # The shape is (batch, height, width, channels). From training, we know this is (1, 128, 431, 1).
    input_shape = input_details[0]['shape']
    dummy_input = np.random.randn(*input_shape).astype(np.float32)

    # Set the tensor and run inference
    interpreter.set_tensor(input_details[0]['index'], dummy_input)
    interpreter.invoke()

    # Get the output and decode the prediction
    output_data = interpreter.get_tensor(output_details[0]['index'])
    prediction = (output_data > 0.5).astype("int32")
    
    if label_encoder:
        predicted_label = label_encoder.inverse_transform(prediction.flatten())
        print(f"Successfully ran inference on '{os.path.basename(model_path)}'.")
        print(f"Input shape: {input_shape}")
        print(f"Output data: {output_data}")
        print(f"Predicted Label: {predicted_label[0]}")
    else:
        print("Cannot decode label without label_encoder.")

# --- Test both models ---
if model:
    print("\\n--- Testing Standard TFLite Model ---")
    test_tflite_model(TFLITE_MODEL_PATH)

    print("\\n--- Testing Quantized TFLite Model ---")
    test_tflite_model(TFLITE_FP16_MODEL_PATH)
else:
    print("Model not loaded, skipping tests.")


\n--- Testing Standard TFLite Model ---
Successfully ran inference on 'water_leak_model.tflite'.
Input shape: [  1 128 431   1]
Output data: [[0.34017965]]
Predicted Label: Leak
\n--- Testing Quantized TFLite Model ---
Successfully ran inference on 'water_leak_model_fp16.tflite'.
Input shape: [  1 128 431   1]
Output data: [[0.34038672]]
Predicted Label: Leak


C:\Users\fe_de\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
C:\Users\fe_de\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
